# Section 1.5: Noisy Simulation

## Modeling realistic quantum errors with noise channels and density matrices

In this section, you'll learn how to simulate realistic quantum hardware imperfections. Real quantum computers suffer from noise—qubits lose energy, lose phase coherence, and experience random errors. Understanding and modeling these noise sources is essential for developing robust quantum algorithms that work on NISQ (Noisy Intermediate-Scale Quantum) devices.

## Learning Objectives

By the end of this section, you will be able to:

- Understand the main types of quantum noise channels (bit flip, depolarizing, amplitude damping, phase damping)
- Use Kraus operators to mathematically describe noise channels
- Distinguish between pure states and mixed states using density matrices
- Simulate noisy quantum circuits with cirq.DensityMatrixSimulator
- Compare ideal and noisy simulation to understand noise impact
- Model T1 (amplitude damping) and T2 (phase damping) decoherence
- Quantify noise effects using purity and fidelity metrics
- Understand how noise degrades entanglement in Bell states

In [ ]:
import cirq
import numpy as np
import matplotlib.pyplot as plt

## 1. Noise Channels

Noise channels model realistic quantum errors. Each channel represents a different physical mechanism that corrupts quantum information. Understanding these channels is crucial for building fault-tolerant quantum algorithms.

In [ ]:
q = cirq.LineQubit(0)

print("Noise channels model realistic quantum errors:\n")

print("1. Bit Flip Channel - random X errors")
print("   Physical interpretation: Qubit spontaneously flips")
bit_flip = cirq.bit_flip(p=0.2)
print(f"   Channel: {bit_flip}")
print(f"   Operation on qubit: {bit_flip(q)}")

print("\n2. Depolarizing Channel - symmetric white noise")
print("   Physical interpretation: Qubit randomly subjected to X, Y, or Z")
depolarize = cirq.depolarize(p=0.1)
print(f"   Channel: {depolarize}")
print(f"   Operation on qubit: {depolarize(q)}")

print("\n3. Amplitude Damping - energy relaxation (T1 decay)")
print("   Physical interpretation: Qubit loses energy to environment")
amp_damp = cirq.amplitude_damp(gamma=0.3)
print(f"   Channel: {amp_damp}")
print(f"   Operation on qubit: {amp_damp(q)}")

print("\n4. Phase Damping - phase relaxation (T2 decay)")
print("   Physical interpretation: Qubit loses phase coherence")
phase_damp = cirq.phase_damp(gamma=0.25)
print(f"   Channel: {phase_damp}")
print(f"   Operation on qubit: {phase_damp(q)}")

## 2. Kraus Operator Representation

Noise channels are mathematically described by Kraus operators {A_k} that satisfy the completeness relation Σ_k A_k† A_k = I. This ensures that the total probability is conserved.

Any noise channel transforms a density matrix as: ρ → Σ_k A_k ρ A_k†

In [ ]:
print("Noise channels are described by Kraus operators {A_k}:")
print("ρ → Σ_k A_k ρ A_k†")
print("\nCompleteness relation: Σ_k A_k† A_k = I\n")

# Bit flip example
print("1. Bit Flip Channel (p=0.2):")
bit_flip = cirq.bit_flip(0.2)
kraus_ops = cirq.kraus(bit_flip)

print(f"   Number of Kraus operators: {len(kraus_ops)}")
for i, K in enumerate(kraus_ops):
    print(f"\n   A_{i}:")
    print(f"   {np.round(K, 3)}")

# Verify completeness
sum_kraus = sum(K.conj().T @ K for K in kraus_ops)
print(f"\n   Completeness check (Σ A_k† A_k):")
print(f"   {np.round(sum_kraus, 6)}")
print(f"   Is identity? {np.allclose(sum_kraus, np.eye(2))}")

# Amplitude damping example
print("\n2. Amplitude Damping Channel (γ=0.3):")
amp_damp = cirq.amplitude_damp(0.3)
kraus_ops = cirq.kraus(amp_damp)

print(f"   Number of Kraus operators: {len(kraus_ops)}")
for i, K in enumerate(kraus_ops):
    print(f"\n   A_{i}:")
    print(f"   {np.round(K, 3)}")

print("\nInterpretation:")
print("  A_0: No-jump operator (no decay occurred)")
print("  A_1: Jump operator (decay from |1⟩ to |0⟩ occurred)")

## 3. Density Matrix Simulation

Density matrices describe both **pure states** (ρ = |ψ⟩⟨ψ|) and **mixed states** (ρ = Σ_i p_i |ψ_i⟩⟨ψ_i|). Unlike state vectors, density matrices can represent statistical mixtures—essential for modeling noise.

A pure state has purity Tr(ρ²) = 1. Mixed states have purity < 1.

In [ ]:
print("Density matrices describe both pure and mixed quantum states")
print("Pure state: ρ = |ψ⟩⟨ψ|")
print("Mixed state: ρ = Σ_i p_i |ψ_i⟩⟨ψ_i|\n")

q = cirq.LineQubit(0)

# Pure state example
print("1. Pure State (|+⟩ superposition):")
pure_circuit = cirq.Circuit(cirq.H(q))

sv_simulator = cirq.Simulator()
sv_result = sv_simulator.simulate(pure_circuit)
psi = sv_result.final_state_vector

dm_simulator = cirq.DensityMatrixSimulator()
dm_result = dm_simulator.simulate(pure_circuit)
rho_pure = dm_result.final_density_matrix

print(f"\n   State vector |ψ⟩:")
print(f"   {np.round(psi, 3)}")
print(f"\n   Density matrix ρ = |ψ⟩⟨ψ|:")
print(f"   {np.round(rho_pure, 3)}")

# Calculate purity
purity_pure = np.trace(rho_pure @ rho_pure).real
print(f"\n   Purity Tr(ρ²) = {purity_pure:.6f}")
print(f"   Pure state has purity = 1")

# Mixed state example
print("\n2. Mixed State (after depolarizing noise):")
mixed_circuit = cirq.Circuit(
    cirq.H(q),
    cirq.depolarize(0.3)(q)
)

dm_result = dm_simulator.simulate(mixed_circuit)
rho_mixed = dm_result.final_density_matrix

print(f"\n   Density matrix (with noise):")
print(f"   {np.round(rho_mixed, 3)}")

purity_mixed = np.trace(rho_mixed @ rho_mixed).real
print(f"\n   Purity Tr(ρ²) = {purity_mixed:.6f}")
print(f"   Mixed state has purity < 1")

# Verify density matrix properties
print("\n3. Density Matrix Properties:")
print(f"   • Hermitian: ρ = ρ†? {np.allclose(rho_mixed, rho_mixed.conj().T)}")
print(f"   • Unit trace: Tr(ρ) = 1? {np.allclose(np.trace(rho_mixed), 1.0)}")
eigenvalues = np.linalg.eigvalsh(rho_mixed)
print(f"   • Positive semi-definite (eigenvalues ≥ 0)? {np.all(eigenvalues >= -1e-10)}")
print(f"   • Eigenvalues: {np.round(eigenvalues, 6)}")

## 4. Ideal vs Noisy Simulation

Comparing ideal and noisy simulations reveals how noise corrupts quantum states. Even small amounts of noise can significantly degrade quantum coherence and entanglement.

In [ ]:
q0, q1 = cirq.LineQubit.range(2)

print("Comparing Bell state preparation with and without noise:\n")

# Ideal Bell state
print("1. IDEAL Simulation:")
ideal_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1)
)

print(f"\nCircuit:")
print(ideal_circuit)

dm_sim = cirq.DensityMatrixSimulator()
ideal_result = dm_sim.simulate(ideal_circuit)
rho_ideal = ideal_result.final_density_matrix

print(f"\nDensity matrix:")
print(np.round(rho_ideal, 3))

purity_ideal = np.trace(rho_ideal @ rho_ideal).real
print(f"\nPurity: {purity_ideal:.6f} (pure state)")

# Noisy Bell state
print("\n2. NOISY Simulation:")
noisy_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.depolarize(0.1)(q0),
    cirq.depolarize(0.1)(q1)
)

print(f"\nCircuit (with noise):")
print(noisy_circuit)

noisy_result = dm_sim.simulate(noisy_circuit)
rho_noisy = noisy_result.final_density_matrix

print(f"\nDensity matrix:")
print(np.round(rho_noisy, 3))

purity_noisy = np.trace(rho_noisy @ rho_noisy).real
print(f"\nPurity: {purity_noisy:.6f} (mixed state)")

print(f"\nPurity reduction: {(1 - purity_noisy/purity_ideal)*100:.1f}%")

# Measurement statistics
print("\n3. Measurement Statistics:")

ideal_circuit_with_measure = ideal_circuit + cirq.Circuit(
    cirq.measure(q0, q1, key='result')
)
noisy_circuit_with_measure = noisy_circuit + cirq.Circuit(
    cirq.measure(q0, q1, key='result')
)

# Run simulations
ideal_run = dm_sim.run(ideal_circuit_with_measure, repetitions=1000)
noisy_run = dm_sim.run(noisy_circuit_with_measure, repetitions=1000)

ideal_counts = ideal_run.histogram(key='result')
noisy_counts = noisy_run.histogram(key='result')

print("\n   Ideal outcomes:")
for outcome in [0, 1, 2, 3]:
    count = ideal_counts.get(outcome, 0)
    print(f"   |{outcome:02b}⟩: {count:4d} ({count/10:.1f}%)")

print("\n   Noisy outcomes:")
for outcome in [0, 1, 2, 3]:
    count = noisy_counts.get(outcome, 0)
    print(f"   |{outcome:02b}⟩: {count:4d} ({count/10:.1f}%)")

print("\nNotice:")
print("  - Ideal: Only |00⟩ and |11⟩ appear (perfect correlation)")
print("  - Noisy: |01⟩ and |10⟩ outcomes emerge (correlation degraded)")

## 5. Amplitude Damping (T1 Decay)

Amplitude damping models energy relaxation—the excited state |1⟩ decays to the ground state |0⟩. This is characterized by the T1 time constant in real quantum hardware.

The damping parameter γ relates to time t and T1 as: γ = 1 - exp(-t/T1)

In [ ]:
print("Amplitude damping models energy relaxation:")
print("Excited state |1⟩ decays to ground state |0⟩\n")

q = cirq.LineQubit(0)

# Prepare excited state
print("1. Prepare |1⟩ state (excited):\n")

gamma_values = [0.0, 0.2, 0.5, 0.8]
dm_sim = cirq.DensityMatrixSimulator()

print("2. Apply amplitude damping with varying γ:")
print(f"\n   γ      |0⟩ pop   |1⟩ pop   Purity")
print(f"   {'─'*40}")

populations = []
for gamma in gamma_values:
    circuit = cirq.Circuit(
        cirq.X(q),
        cirq.amplitude_damp(gamma)(q)
    )

    result = dm_sim.simulate(circuit)
    rho = result.final_density_matrix

    pop_0 = rho[0, 0].real
    pop_1 = rho[1, 1].real
    purity = np.trace(rho @ rho).real

    populations.append((pop_0, pop_1))

    print(f"   {gamma:.1f}    {pop_0:.3f}    {pop_1:.3f}    {purity:.3f}")

print("\n   As γ increases:")
print("   • |0⟩ population increases (energy loss)")
print("   • |1⟩ population decreases")
print("   • State becomes mixed (purity < 1)")
print("\n   Physical meaning: Qubit relaxes to ground state over time")

## 6. Phase Damping (T2 Decay)

Phase damping models loss of phase coherence without energy loss. Superposition states lose their off-diagonal density matrix elements, eventually becoming classical statistical mixtures.

This is characterized by the T2 dephasing time in real quantum hardware. Note: T2 ≤ 2T1 always.

In [ ]:
print("Phase damping models loss of phase coherence:")
print("Superposition states lose off-diagonal density matrix elements\n")

q = cirq.LineQubit(0)

# Prepare superposition
print("1. Prepare |+⟩ = (|0⟩ + |1⟩)/√2:\n")

gamma_values = [0.0, 0.2, 0.5, 0.8]
dm_sim = cirq.DensityMatrixSimulator()

print("2. Apply phase damping with varying γ:")
print(f"\n   γ      ρ₀₀     ρ₁₁     |ρ₀₁|    Purity")
print(f"   {'─'*50}")

for gamma in gamma_values:
    circuit = cirq.Circuit(
        cirq.H(q),
        cirq.phase_damp(gamma)(q)
    )

    result = dm_sim.simulate(circuit)
    rho = result.final_density_matrix

    rho_00 = rho[0, 0].real
    rho_11 = rho[1, 1].real
    rho_01 = abs(rho[0, 1])
    purity = np.trace(rho @ rho).real

    print(f"   {gamma:.1f}    {rho_00:.3f}   {rho_11:.3f}   {rho_01:.3f}   {purity:.3f}")

print("\n   As γ increases:")
print("   • Diagonal elements (populations) stay constant")
print("   • Off-diagonal elements (coherences) decay to zero")
print("   • State becomes classically mixed")
print("\n   Physical meaning: Quantum coherence lost but energy preserved")

## 7. Visualizing Noise Effects on Density Matrices

Visualizing density matrices helps understand how different noise channels affect quantum states. Color intensity shows the magnitude of matrix elements.

In [ ]:
q = cirq.LineQubit(0)

# Prepare superposition state
base_circuit = cirq.Circuit(cirq.H(q))

noise_channels = [
    ("Bit Flip", cirq.bit_flip(0.3)),
    ("Depolarize", cirq.depolarize(0.3)),
    ("Amplitude Damp", cirq.amplitude_damp(0.3)),
    ("Phase Damp", cirq.phase_damp(0.3))
]

dm_sim = cirq.DensityMatrixSimulator()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Ideal state
ideal_result = dm_sim.simulate(base_circuit)
rho_ideal = ideal_result.final_density_matrix

# Plot ideal density matrix (real and imaginary parts)
im0 = axes[0].imshow(np.real(rho_ideal), cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_title("Ideal State\n(Real part)", fontsize=12, fontweight='bold')
axes[0].set_xlabel('Column')
axes[0].set_ylabel('Row')
plt.colorbar(im0, ax=axes[0])

axes[1].imshow(np.imag(rho_ideal), cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_title("Ideal State\n(Imaginary part)", fontsize=12, fontweight='bold')
axes[1].set_xlabel('Column')
axes[1].set_ylabel('Row')
plt.colorbar(im0, ax=axes[1])

# Plot each noise channel
for idx, (name, channel) in enumerate(noise_channels):
    circuit = base_circuit + cirq.Circuit(channel(q))
    result = dm_sim.simulate(circuit)
    rho = result.final_density_matrix

    # Plot real part
    ax_idx = idx + 2
    im = axes[ax_idx].imshow(np.real(rho), cmap='RdBu', vmin=-1, vmax=1)
    purity = np.trace(rho @ rho).real
    axes[ax_idx].set_title(f"{name}\nPurity: {purity:.3f}",
                           fontsize=12, fontweight='bold')
    axes[ax_idx].set_xlabel('Column')
    axes[ax_idx].set_ylabel('Row')
    plt.colorbar(im, ax=axes[ax_idx])

plt.suptitle("Density Matrix Real Parts: Ideal vs Noisy States",
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()

print("Displaying density matrix visualization...")
plt.savefig('noisy_simulation_density_matrices.png', dpi=150, bbox_inches='tight')
print("  Saved to: noisy_simulation_density_matrices.png")
plt.show()

print("\nKey observations:")
print("  - Ideal: Strong off-diagonal elements (quantum coherence)")
print("  - Phase damp: Off-diagonal elements disappear (coherence loss)")
print("  - Amplitude damp: Diagonal elements change (population change)")
print("  - Depolarize: Both diagonal and off-diagonal affected (total randomization)")

## 8. Purity vs Noise Strength

Purity Tr(ρ²) quantifies how mixed a quantum state is. Pure states have purity = 1, while maximally mixed states approach purity = 1/d (where d is the dimension).

This visualization shows how different noise channels degrade purity at different rates.

In [ ]:
q = cirq.LineQubit(0)
base_circuit = cirq.Circuit(cirq.H(q))

# Vary noise parameter
noise_params = np.linspace(0, 0.99, 50)

channels = {
    'Bit Flip': lambda p: cirq.bit_flip(p),
    'Depolarize': lambda p: cirq.depolarize(p),
    'Amplitude Damp': lambda p: cirq.amplitude_damp(p),
    'Phase Damp': lambda p: cirq.phase_damp(p)
}

dm_sim = cirq.DensityMatrixSimulator()

plt.figure(figsize=(10, 6))

print("Computing purity degradation for different noise channels...\n")

for name, channel_func in channels.items():
    purities = []

    for p in noise_params:
        circuit = base_circuit + cirq.Circuit(channel_func(p)(q))
        result = dm_sim.simulate(circuit)
        rho = result.final_density_matrix
        purity = np.trace(rho @ rho).real
        purities.append(purity)

    plt.plot(noise_params, purities, label=name, linewidth=2, marker='o',
            markersize=3, markevery=5)
    
    print(f"  {name}: purity at p=0.5 is {purities[25]:.3f}")

plt.xlabel('Noise Parameter (p or γ)', fontsize=12)
plt.ylabel('Purity Tr(ρ²)', fontsize=12)
plt.title('Purity Degradation with Increasing Noise', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xlim(0, 1)
plt.ylim(0, 1.05)

# Add reference line for pure state
plt.axhline(y=1.0, color='black', linestyle='--', alpha=0.3, label='Pure state')

plt.tight_layout()

print("\nDisplaying purity vs noise strength plot...")
plt.savefig('purity_vs_noise.png', dpi=150, bbox_inches='tight')
print("  Saved to: purity_vs_noise.png")
plt.show()

print("\nKey observations:")
print("  - All noise channels degrade purity monotonically")
print("  - Depolarizing noise is most destructive to purity")
print("  - Phase damping and bit flip show similar purity loss")
print("  - Maximally mixed state has purity = 0.5 for single qubit")

## 9. Bell State Fidelity Under Noise

Fidelity F(ρ, σ) measures how similar two quantum states are. For a pure state |ψ⟩ and density matrix ρ, the fidelity simplifies to F = |⟨ψ|ρ|ψ⟩|.

This shows how noise degrades the quality of entangled Bell states—critical for quantum communication and error correction.

In [ ]:
print("Fidelity F(ρ, σ) = Tr(√(√ρ σ √ρ))² measures state similarity")
print("Perfect fidelity = 1, completely different = 0\n")

q0, q1 = cirq.LineQubit.range(2)

# Ideal Bell state
ideal_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1)
)

dm_sim = cirq.DensityMatrixSimulator()
ideal_result = dm_sim.simulate(ideal_circuit)
rho_ideal = ideal_result.final_density_matrix

# Vary noise strength
noise_levels = np.linspace(0, 0.5, 30)
fidelities_depol = []
fidelities_amp_damp = []

print("Computing Bell state fidelity degradation...\n")

for noise_p in noise_levels:
    # Depolarizing noise
    noisy_circuit_depol = cirq.Circuit(
        cirq.H(q0),
        cirq.CNOT(q0, q1),
        cirq.depolarize(noise_p)(q0),
        cirq.depolarize(noise_p)(q1)
    )
    result_depol = dm_sim.simulate(noisy_circuit_depol)
    rho_noisy_depol = result_depol.final_density_matrix

    # Fidelity calculation (valid when rho_ideal is pure): F = |Tr(ρ†σ)|
    # For general mixed states, use full formula: F(ρ, σ) = Tr(√(√ρ σ √ρ))²
    fidelity_depol = np.abs(np.trace(rho_ideal.conj().T @ rho_noisy_depol))
    fidelities_depol.append(fidelity_depol)

    # Amplitude damping
    noisy_circuit_amp = cirq.Circuit(
        cirq.H(q0),
        cirq.CNOT(q0, q1),
        cirq.amplitude_damp(noise_p)(q0),
        cirq.amplitude_damp(noise_p)(q1)
    )
    result_amp = dm_sim.simulate(noisy_circuit_amp)
    rho_noisy_amp = result_amp.final_density_matrix

    # Fidelity calculation (valid when rho_ideal is pure)
    fidelity_amp = np.abs(np.trace(rho_ideal.conj().T @ rho_noisy_amp))
    fidelities_amp_damp.append(fidelity_amp)

print(f"  Fidelity with depolarizing noise (p=0.1): {fidelities_depol[6]:.3f}")
print(f"  Fidelity with amplitude damping (γ=0.1): {fidelities_amp_damp[6]:.3f}\n")

plt.figure(figsize=(10, 6))
plt.plot(noise_levels, fidelities_depol, label='Depolarizing Noise',
        linewidth=2, marker='o', markersize=4)
plt.plot(noise_levels, fidelities_amp_damp, label='Amplitude Damping',
        linewidth=2, marker='s', markersize=4)

plt.xlabel('Noise Parameter', fontsize=12)
plt.ylabel('Fidelity with Ideal Bell State', fontsize=12)
plt.title('Bell State Degradation Under Different Noise Models',
         fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xlim(0, 0.5)
plt.ylim(0, 1.05)

plt.tight_layout()

print("Displaying Bell state fidelity plot...")
plt.savefig('bell_state_fidelity.png', dpi=150, bbox_inches='tight')
print("  Saved to: bell_state_fidelity.png")
plt.show()

print("\nKey observations:")
print("  - Depolarizing noise degrades fidelity faster than amplitude damping")
print("  - Bell state entanglement is fragile under noise")
print("  - Even 10% noise can significantly reduce fidelity")
print("  - This motivates quantum error correction research")

## Exercises

1. **Custom Noise Model**: Create a circuit with a combination of amplitude damping (γ=0.1) and phase damping (γ=0.15). Compare the purity to each noise channel applied individually. Which effect dominates?

2. **T1/T2 Relationship**: Real hardware has the constraint T2 ≤ 2T1. Create circuits with amplitude and phase damping that violate and satisfy this constraint. What happens to the density matrix in each case?

3. **Multi-Qubit Noise**: Create a 3-qubit GHZ state and apply depolarizing noise (p=0.05) to each qubit. Calculate the fidelity with the ideal GHZ state. How does it compare to the 2-qubit Bell state fidelity?

4. **Kraus Operator Exploration**: Manually implement a custom noise channel using Kraus operators. Verify it satisfies the completeness relation and preserves trace.

5. **Noise Mitigation**: Design a simple noise mitigation strategy. For example, repeat a circuit multiple times and average the results. Does this improve fidelity?

6. **Mixed vs Pure**: Create two circuits that produce the same diagonal density matrix elements but different purity. Explain the physical difference between these states.

## Key Takeaways

- **Noise channels** model realistic quantum errors: bit flip, depolarizing, amplitude damping (T1), and phase damping (T2)
- **Kraus operators** provide a mathematical description of noise channels satisfying Σ_k A_k† A_k = I
- **Density matrices** ρ describe both pure states (purity = 1) and mixed states (purity < 1)
- **cirq.DensityMatrixSimulator** enables simulation of noisy quantum circuits
- **Amplitude damping** models energy relaxation (|1⟩ → |0⟩) characterized by T1 time
- **Phase damping** models coherence loss without energy loss, characterized by T2 time
- **Purity** Tr(ρ²) quantifies how mixed a state is, with 1 = pure and < 1 = mixed
- **Fidelity** measures how close a noisy state is to the ideal state
- **Entanglement is fragile**: Even small noise significantly degrades Bell states and other entangled states
- Understanding noise is essential for developing robust NISQ algorithms and error correction protocols

---

**Next:** Section 1.6 - Parameterized Circuits - Learn to create parameterized circuits for variational quantum algorithms